In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
model = tf.keras.models.load_model(".../models/TCN0.keras")

First we load the train set and get its means and stdevs

In [ ]:
anom_train = pd.read_parquet(".../data/TEP_Faulty_Training.parquet")

In [4]:
X_train = anom_train.drop(["simulationRun","sample","faultNumber"],axis=1)

In [5]:
del anom_train

In [6]:
X_train = X_train.values.reshape((10000, 500, 52))

In [7]:
# Calculate means and standard deviations from the training set
train_means = X_train.mean(axis=(0, 1))
train_std = X_train.std(axis=(0, 1))

In [8]:
del X_train

In [ ]:
anom_test = pd.read_parquet(".../data/TEP_Faulty_Testing.parquet")

In [10]:
anom_test.shape

(9600000, 55)

In [11]:
anom_test.shape[0] / 960

10000.0

In [12]:
anom_test.shape[0] / 960 / 20

500.0

In [13]:
# Preparing test set
simulation_length = 960
num_simulations = 10000
num_features = 52
num_classes = 20

y_test = anom_test['faultNumber'].values[::simulation_length]
X_test = anom_test.drop(["simulationRun","sample","faultNumber"],axis=1)

In [14]:
del anom_test

In [15]:
X_test = X_test.values.reshape((num_simulations, simulation_length, num_features))

In [16]:
X_test.shape

(10000, 960, 52)

In [17]:
y_test.shape

(10000,)

In [18]:
# Standardize the test set
X_test_scaled = (X_test - train_means) / (train_std)

In [19]:
y_test_encoded = tf.keras.utils.to_categorical(y_test - 1, num_classes=num_classes)

In [20]:
loss, acc = model.evaluate(X_test_scaled, y_test_encoded)

313/313 ━━━━━━━━━━━━━━━━━━━━ 53s 150ms/step - accuracy: 0.9312 - loss: 0.3279


In [21]:
loss

0.32733577489852905

In [22]:
acc

0.9301999807357788

----------